http://sebastianraschka.com/Articles/2014_python_lda.html#step-1-computing-the-d-dimensional-mean-vectors

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('flies.txt',delimiter='\t')
# df = df.reindex_axis(sorted(df.columns), axis=1)

# group column
group = ['fm']

# covariance given by the lecturer
V = np.array([[0.288089, 0.186644], [0.186644, 0.164457]])
y = np.array([4.0, 2.1])
df

,length,width,wings,e,fm
0,3.30,NaN,2.45,1.45,0
1,3.71,0.93,2.58,1.75,0
2,3.75,0.95,2.53,1.50,0
3,4.05,NaN,NaN,2.00,0
4,4.14,1.10,2.77,2.00,0
5,4.25,0.97,3.00,1.94,0
6,4.45,NaN,2.71,2.05,0
7,4.60,1.15,3.10,2.20,0
8,4.72,1.22,3.10,2.16,0
9,5.00,1.19,2.95,2.25,0


In [3]:
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

p = partition(df.fm.values)
df.values[p[0]]

array([[ 3.3 ,   nan,  2.45,  1.45,  0.  ],
       [ 3.71,  0.93,  2.58,  1.75,  0.  ],
       [ 3.75,  0.95,  2.53,  1.5 ,  0.  ],
       [ 4.05,   nan,   nan,  2.  ,  0.  ],
       [ 4.14,  1.1 ,  2.77,  2.  ,  0.  ],
       [ 4.25,  0.97,  3.  ,  1.94,  0.  ],
       [ 4.45,   nan,  2.71,  2.05,  0.  ],
       [ 4.6 ,  1.15,  3.1 ,  2.2 ,  0.  ],
       [ 4.72,  1.22,  3.1 ,  2.16,  0.  ],
       [ 5.  ,  1.19,  2.95,  2.25,  0.  ],
       [ 5.11,  1.43,  3.6 ,  2.63,  0.  ],
       [ 5.2 ,  1.42,  3.32,  2.6 ,  0.  ]])

## Compute group means

In [4]:
features = ['length', 'e']

means = df[features + group].dropna().groupby(group).mean()

# seperate group means 
# mu0 = 'length' and 'e' mean for males
# mu1 = 'length' and 'e' mean for females
mu0, mu1 = means.loc[0], means.loc[1]

print(mu0)
print( mu1)

length    4.356667
e         2.044167
Name: 0, dtype: float64
length    4.124444
e         2.473333
Name: 1, dtype: float64


## Cov matrix
    with Pandas groupby below we use another method since this doesn't return consistent results
    for more info see last section "Troubleshooting"

In [5]:
group_sizes = df[features + group].dropna().groupby(group).count().T.iloc[0]
group_sizes

fm
0    12
1     9
Name: length, dtype: int64

In [6]:
tmp = df[features].dropna()
tmp = pd.concat([tmp, df[group]], axis=1)
c = tmp.groupby(group).cov()
group_sizes = tmp.groupby(group).count().T.iloc[0]
n_groups = group_sizes.size
n = len(tmp)
cov_mat = np.multiply(c.values.reshape(n_groups, - 1),group_sizes.values[:, np.newaxis] - 1).sum(axis=0) / (n - n_groups)
pd.DataFrame(cov_mat.reshape(c.shape[1], c.shape[1]))

,0,1
0,0.164457,0.186644
1,0.186644,0.288089


## Mahalanobis distance

In [7]:
def mahalanobis(y, mu, V):
    """Compute the mahalanobis distance.
    
    Paramaters
    ----------
    y : np.array-like
        Some observation.
    
    mu : np.array-like
        Group mean vector.
        
    V : np.array-like
        Covariance matrix.
        
    Returns
    -------
        Mahalanobis distance.
    """
    return (y-mu).dot(V).dot(y-mu)

In [8]:
print('Mahalanobis distance with respect to group 0:', mahalanobis(y, mu0, V))
print('Mahalanobis distance with respect to group 1:', mahalanobis(y, mu1, V))

Mahalanobis distance with respect to group 0: 0.0297271778896
Mahalanobis distance with respect to group 1: 0.0447258023506


Mahalanobis distance is smaller for group 0 so we assume that observed object belongs to group 0.

## Classify data
    For each observation (row) we compute the mahalanobis distance
    with respect to each group and classify the data according to the min value

In [9]:
from collections import defaultdict

# save distances as dict of dicts
# so we can easily construct a pandas 
# DF afterwards
m_dist = defaultdict(dict)


for row in df.index:
    # grab observation
    obs = df.iloc[row][features]
    
    # compute its distance to each group
    m_dist['maha_0'][row] = mahalanobis(obs, mu0, V)
    m_dist['maha_1'][row] = mahalanobis(obs, mu1, V)
    
df_class = pd.DataFrame(m_dist)

# classify obs as group 1 if its distance is smaller
cond = df_class['maha_1'] < df_class['maha_0']
df_class['class'] = cond.astype(int)
df_class

,maha_0,maha_1,class
0,0.614087,0.682974,0
1,0.205713,0.247434,0
2,0.277961,0.332244,0
3,0.032470,0.051596,0
4,0.017417,0.034167,0
5,0.009210,0.026324,0
6,0.002718,0.008560,0
7,0.035207,0.028917,1
8,0.055948,0.048669,1
9,0.175632,0.156058,1


## Accuracy score

In [10]:
def accuracy(y, y_hat):
    """Compute classification accuracy.
    
    Parameters
    ----------
    y : array-like
        True classes.
    
    y_hat : array-like
        Predicted classes.
        
    Returns
    -------
    accuracy : float
        Relative frequency of correct classifications.
    """
    return sum(y == y_hat) / y.size

In [11]:
print('Accuracy score of discriminant analysis on the data set: %.2f' %
      (accuracy(df[group].values.ravel(), df_class['class'].values.ravel()) * 100))

Accuracy score of discriminant analysis on the data set: 61.90


In [14]:
df[features].cov()

,length,e
length,0.287551,0.151684
e,0.151684,0.203596


## Pack everything into a class so it's easier to solve task 2

In [ ]:
# %load LDA.py
# License AGPL-3.0 or later (http://www.gnu.org/licenses/agpl.html).
import numpy as np
import pandas as pd


class LDA(object):
    def __init__(self, df, group):
        self.df = df
        self.group = [group]

    def compute_group_means(self, features=None, verbose=0):
        """Compute group means.

        Parameters
        ----------
        features : list of str
            String of list containing the features for which to compute
             the group means.

        verbose : bool, optional
            If 1 or True, the group mean vectors will be printed as DataFrame.

        Returns
        -------
        group_mean_vectors : pd.DataFrame
            DataFrame where each row corresponds to one group mean vector.
            Groups are indicated by the index.
        """
        df = self.df
        if features:
            df = df[features + self.group]

        self.group_mean_vectors = df.dropna().groupby(self.group).mean()

        if verbose:
            print("Group mean vectors:\n", self.group_mean_vectors)
        return self.group_mean_vectors

    def mahalanobis(self, y, mu, cov=None):
        """Compute the mahalanobis distance.

        For a given observation y, compute its mahalanobis distance to the
        vector mu with respect to the covariance matrix V.

        Paramaters
        ----------
        y : np.array-like
            Some observation. Must be of same dimension as mu.

        mu : np.array-like
            Group mean vector.

        V : np.array-like
            Covariance matrix.

        Returns
        -------
            Mahalanobis distance.
        """
        if not hasattr(self, 'cov'):
            self.cov = self.compute_cov()
            self.cov_inv = np.linalg.inv(self.cov)

        return (y - mu).dot(self.cov_inv).dot(y - mu)

    def accuracy(self, y, y_hat):
        """Compute classification accuracy.

        Parameters
        ----------
        y : array-like
            True classes.

        y_hat : array-like
            Predicted classes.

        Returns
        -------
        accuracy : float
            Relative frequency of correct classifications.
        """
        return sum(y == y_hat) / y.size

    def compute_cov(self, features=None):
        """Return covariance matrix for given features.

        Parameters
        ----------
        features: list of str, optional
            List of features for which to return the
            corresponding covariance.

        Returns
        -------
        cov: pd.DataFrame
        """
        if not features:
            tmp = self.df.dropna()
        else:
            tmp = self.df[features + self.group].dropna()

        self.group_sizes = tmp.groupby(self.group, sort=False).count().T.iloc[
            0]
        self.group_sizes.name = 'group_sizes'
        self.n_groups = self.group_sizes.size
        n = len(tmp)

        c = tmp.groupby(self.group, sort=False).cov()
        self.c = c
        cov_mat = np.multiply(
            c.values.reshape(self.n_groups, - 1),
            self.group_sizes.values[:, np.newaxis] - 1).sum(axis=0) / \
                  (n - self.n_groups)
        self.tmp = tmp
        self.cov = pd.DataFrame(cov_mat.reshape(c.shape[1], c.shape[1]))
        self.cov_inv = np.linalg.inv(self.cov)

        return self.cov

    def partition(self, a):
        """Partition array into its unique classes.

        This method takes an array as input and returns a dictionary
        that maps each unique value to its indices.

        Parameters
        ----------
        a : np.array
            Array which consists of numerical (int) classes.

        Returns
        -------
        Dictionary that maps each unique value of the array to its indices.
        """
        return {c: (a == c).nonzero()[0] for c in np.unique(a)}

    def compute_S_W(self, features=None, verbose=0):
        """Compute within-groups scatter matrix.

        A pxp within-groups scatter matrix is computed where p is the number
        of features being considered. Missing values will be dropped before
        computing the actual scatter matrix.

        .. math:: S_w = \\sum_{i=1}^c S_i

        with `S_i` being the scatter-matrix of each group G_i (with group
        size n_i):

        .. math:: S_i = \\sum_{x \in G_i}^n_i (x - \mu_i) * (x - \mu_i)^T

        and `m_i` the group mean vector:

        .. math:: \frac{1}{n_i} \\sum_{x \in G_i}^n_i x

        Parameters
        ----------
        features : list of str, optional
            If provided, computation is based only on a data subset consisting
            of the given features.

        verbose : bool, optional
            If 1 or True, the within-groups scatter matrix will be printed.

        Returns
        -------
        S_W : array-like
            Within-groups scatter matrix.
        """
        X, y = self.prepare_data(features)

        # number of features
        p = self.p = X.shape[1]

        # number of total obs
        self.n_total = len(y)

        # number of groups
        self.n_groups = np.unique(y).shape[0]

        self.S_W = np.zeros((p, p))
        self.group_mean_vectors = self.compute_group_means(features, verbose)

        for klass, mean_vec in self.group_mean_vectors.iterrows():
            #  Compute group mean difference x - mu_p
            dif = X[y == klass] - mean_vec.values.reshape(-1, )
            self.S_W += np.tensordot(dif, dif, axes=((0), (0)))

        if verbose:
            print('within-groups Scatter Matrix:\n', pd.DataFrame(self.S_W))

        return self.S_W

    def compute_S_B(self, features=None, verbose=0):
        """Compute between-groups scatter matrix.

        A pxp between-groups scatter matrix is computed where p is the number
        of features being considered. Missing values will be dropped before
        computing the actual scatter matrix.

        The computation is based on the following equation:

        .. math:: S_b = \\sum_{i=1}^c N_i * (\mu_i - \mu) * (\mu_i - \mu)^T

        where `c` is the number of classes, `mu_i` and `N_i` the mean and
        size of the respective classes and `mu` the overall mean.

        Parameters
        ----------
        features : list of str, optional
            If provided, computation is based only on a data subset consisting
            of the given features.

        verbose : bool, optional
            If 1 or True, the between-groups scatter matrix will be printed.

        Returns
        -------
        S_B : array-like
            Between-groups scatter matrix.
        """
        X, y = self.prepare_data(features)

        # number of features
        p = self.p = X.shape[1]

        self.group_mean_vectors = self.compute_group_means(features, verbose)
        overall_mean = self.overall_mean = np.mean(X, axis=0)

        self.S_B = np.zeros((p, p))
        for klass, mean_vec in self.group_mean_vectors.iterrows():
            n = X[y == klass, :].shape[0]
            mean_vec = mean_vec.values.reshape(4, 1)  # make column vector
            overall_mean = overall_mean.reshape(4, 1)  # make column vector
            self.S_B += n * (mean_vec - overall_mean).dot(
                (mean_vec - overall_mean).T)
        if verbose:
            print('between-groups Scatter Matrix:\n', pd.DataFrame(self.S_B))

        return self.S_B

    def solve(self, verbose=1):
        """Solve the generalized eigenvalue problem.

        Parameters
        ----------
        verbose

        Returns
        -------

        """
        if not hasattr(self, 'S_B'):
            self.compute_S_B(verbose=verbose)
        if not hasattr(self, 'S_W'):
            self.compute_S_W(verbose=verbose)

        eig_vals, eig_vecs = np.linalg.eig(
            np.linalg.inv(self.S_W).dot(self.S_B))

        if verbose:
            for i in range(len(eig_vals)):
                eigvec_sc = eig_vecs[:, i].reshape(4, 1)
                print('\nEigenvector {}: \n{}'.format(i + 1, eigvec_sc.real))
                print('Eigenvalue {:}: {:.2e}'.format(i + 1, eig_vals[i].real))

        self.eig_vals, self.eig_vecs = eig_vals, eig_vecs

    def prepare_data(self, features=None):
        """Prepare data by splitting it into X and y matrices.

        Rows with missing values will be droped.

        Parameters
        ----------
        features : list of str, optional
            If provided, data will be split into a subset of the given
            features.

        Returns
        -------
        X, y : np.array
            X : Matrix consisting of the dependent variables
            y : Vector containing the groups labels.
        """
        df = self.df.dropna()
        if features:
            df = self.df[features + self.group].dropna()

        self.X = df.drop(self.group, axis=1).values
        self.y = df[self.group].values.ravel()

        return self.X, self.y

    def compute_pooled_cov(self, features=None, verbose=0):
        """Compute pooled covariance matrix.

        Parameters
        ----------
        features : list of str, optional
            If provided, computation is based only on a data subset consisting
            of the given features.

        verbose : bool, optional
            If 1 or True, the within-groups scatter matrix might be printed.

        Returns
        -------
        S : array-like
            Pooled covariance matrix.
        """
        if not hasattr(self, 'S_W') or features:
            self.S_W = self.compute_S_W(features, verbose)

        self.cov = self.S_W / (self.n_total - self.n_groups)

        if verbose:
            print("Pooled covariance matrix:\n", pd.DataFrame(self.cov))

        return self.cov

    def sanity_check(self):
        """Sanity check for the solution of the eigenvalue problem."""
        if not hasattr(self, 'eig_vals'):
            raise AttributeError('No eigenvalues found. Please compute them '
                                 'first.')

        for i in range(len(self.eig_vals)):
            eigv = self.eig_vecs[:, i].reshape(self.p, 1)
            np.testing.assert_array_almost_equal(
                np.linalg.inv(self.S_W).dot(self.S_B).dot(eigv),
                self.eig_vals[i] * eigv,
                decimal=6, err_msg='', verbose=True)
        print('Sanity check passed.')


### Load LDA class

In [23]:
%run LDA.py

### Compute eigenvectors and their eigenvalues

In [24]:
lda = LDA(df, 'fm')

# compute within-class scatter matrix
lda.compute_S_W(verbose=1)
print()

# show pooled covariance matrix even though we don't need it here
lda.compute_pooled_cov(verbose=1)
print()

# compute between-class scatter matrix
lda.compute_S_B(verbose=1)
print()

# solve the generalized eigenvalue problem
lda.solve()
print()

# sanity check
lda.sanity_check()

Group mean vectors:
       length     width     wings         e
fm                                        
0   4.497778  1.151111  2.994444  2.114444
1   4.045714  1.138571  2.802857  2.437143
within-groups Scatter Matrix:
           0         1         2         3
0  3.879927  1.077579  2.124675  2.725303
1  1.077579  0.356975  0.660984  0.843627
2  2.124675  0.660984  1.515365  1.880379
3  2.725303  0.843627  1.880379  2.663965

Pooled covariance matrix:
           0         1         2         3
0  0.277138  0.076970  0.151762  0.194665
1  0.076970  0.025498  0.047213  0.060259
2  0.151762  0.047213  0.108240  0.134313
3  0.194665  0.060259  0.134313  0.190283

Group mean vectors:
       length     width     wings         e
fm                                        
0   4.497778  1.151111  2.994444  2.114444
1   4.045714  1.138571  2.802857  2.437143
between-groups Scatter Matrix:
           0         1         2         3
0  0.804673  0.022321  0.341025 -0.574403
1  0.022321  0.000

In [19]:
mean_vectors = lda.compute_group_means()#['length', 'e'])
mean_vectors

,length,width,wings,e
fm,,,,
0,4.497778,1.151111,2.994444,2.114444
1,4.045714,1.138571,2.802857,2.437143


In [25]:
lda.compute_pooled_cov()#['length', 'e'])


,0,1,2,3
0,0.277138,0.076970,0.151762,0.194665
1,0.076970,0.025498,0.047213,0.060259
2,0.151762,0.047213,0.108240,0.134313
3,0.194665,0.060259,0.134313,0.190283


lda.mahalanobis(4, mu0), lda.mahalanobis(2.1, mu1)

### Trouble shooting
    Cov matrix order is somehow broken when we filter columns before applying groupby().cov()

In [15]:
df.dropna().groupby(group).cov()

e    length     width     wings
fm                                               
0  e       0.135403  0.196786  0.065532  0.117403
   length  0.196786  0.318644  0.097753  0.170499
   width   0.065532  0.097753  0.035136  0.057169
   wings   0.117403  0.170499  0.057169  0.116853
1  e       0.263457  0.191836  0.053229  0.156860
   length  0.191836  0.221795  0.049260  0.126781
   width   0.053229  0.049260  0.012648  0.033938
   wings   0.156860  0.126781  0.033938  0.096757

In [16]:
df.head()

,length,width,wings,e,fm
0,3.30,NaN,2.45,1.45,0
1,3.71,0.93,2.58,1.75,0
2,3.75,0.95,2.53,1.50,0
3,4.05,NaN,NaN,2.00,0
4,4.14,1.10,2.77,2.00,0


In [17]:
t = df[features + group].dropna()
t.head()

,length,e,fm
0,3.30,1.45,0
1,3.71,1.75,0
2,3.75,1.50,0
3,4.05,2.00,0
4,4.14,2.00,0


### Now if we use column indexing

In [18]:
t.groupby('fm')[features].cov()

length         e
fm                           
0  length  0.358879  0.209588
   e       0.209588  0.134790
1  length  0.190753  0.155096
   e       0.155096  0.205250

In [19]:
t.groupby('fm').cov()

e    length
fm                           
0  e       0.134790  0.209588
   length  0.209588  0.358879
1  e       0.205250  0.155096
   length  0.155096  0.190753

**The order above is not the same**

In [583]:
df.groupby(group)[features].cov()

length         e
fm                           
0  length  0.358879  0.209588
   e       0.209588  0.134790
1  length  0.190753  0.155096
   e       0.155096  0.205250

In [582]:
df[features + group].groupby(group).cov()

e    length
fm                           
0  e       0.134790  0.209588
   length  0.209588  0.358879
1  e       0.205250  0.155096
   length  0.155096  0.190753

**But why?**